In [401]:
import pandas as pd
import numpy as np
import h5py

import os
import sys
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [402]:
def get_non_dominated_points(array: np.array, columns_to_negate: list):
    # Negate columns for minimisation
    for col in columns_to_negate:
        array[:, col] = - array[:, col]
    # Get non_dominated points
    result = non_dominated(array)
    # Negate back the columns
    for col in columns_to_negate:
        array[:, col] = - array[:, col]
        result[:, col] = - result[:, col]
    return result

In [403]:
from converter import Converter
from core.learners.metrics import non_dominated

# Objectives
objectives = ["egypt_irr", "egypt_low_had", "sudan_irr", "ethiopia_hydro"]

# Read the solutions from EMODPS
logdir_emodps = "output_data/baseline_results_emodps.csv"
emodps_points = pd.read_csv(logdir_emodps, usecols=objectives).to_numpy()

results_emodps = get_non_dominated_points(emodps_points, columns_to_negate=[0, 1, 2])
# Change back to pandas
results_emodps = pd.DataFrame(results_emodps, columns=objectives)

# Read the solutions from MONES
logdir_mones = "output_data/baseline_results_mones.h5"
with h5py.File(logdir_mones, "r") as f:
    nd_points = non_dominated(f["train"]["returns"]["ndarray"][-1])
    # Convert the solution to fit EMODPS data
    nd_points_converted = Converter.convert_array(nd_points)
    results_mones = pd.DataFrame(nd_points_converted, columns=objectives)
    f.close()

In [404]:
print("EMODPS non-dominated points:\n", results_emodps)
print(f"There are {len(results_emodps)} non-dominated points.")

EMODPS non-dominated points:
      egypt_irr  egypt_low_had  sudan_irr  ethiopia_hydro
0     5.844010       0.241667   0.079331       15.009673
1     5.370892       0.137500   0.000000       14.266241
2     5.994473       0.208333   0.072954       15.009900
3     4.551618       0.362500   0.000000       14.304393
4     5.092632       0.191667   0.000000       14.236476
..         ...            ...        ...             ...
217   3.527874       0.187500   0.036244       11.636790
218   4.411613       0.391667   0.061653       14.827489
219   4.890457       0.258333   0.094999       14.896243
220  17.844476       0.000000   0.554119       15.100161
221   4.741431       0.275000   0.072363       14.926539

[222 rows x 4 columns]
There are 222 non-dominated points.


In [405]:
print("MONES non-dominated points:\n", results_mones)
print(f"There are {len(results_mones)} non-dominated points.")

MONES non-dominated points:
     egypt_irr  egypt_low_had  sudan_irr  ethiopia_hydro
0    7.050511       0.000000   0.000000        3.852477
1    6.501449       0.058333   0.000000        3.810921
2    4.712152       0.775000   0.030508        3.146889
3    6.911745       0.016667   0.000000        3.852477
4    7.048976       0.000000   0.000000        3.721112
5    5.406947       0.687500   0.000000        3.764143
6    6.125184       0.304167   0.000000        3.852476
7    5.993262       0.308333   0.000000        3.652694
8    5.118872       0.712500   0.000000        3.758328
9    6.120017       0.337500   0.000000        3.826333
10   4.855628       0.750000   0.000580        3.504006
11   6.870209       0.025000   0.000000        3.852477
12   6.019737       0.487500   0.000000        3.686051
13   6.368688       0.058333   0.000000        3.677921
There are 14 non-dominated points.


In [406]:
print("Objective \t\t Best MONES \t\t Best EMODPS")
print(f"egypt_irr \t\t {results_mones['egypt_irr'].min()} \t\t {results_emodps['egypt_irr'].min()}")
print(f"egypt_low_had \t {results_mones['egypt_low_had'].min()} \t\t\t\t {results_emodps['egypt_low_had'].min()}")
print(f"sudan_irr \t\t {results_mones['sudan_irr'].min()} \t\t\t\t {results_emodps['sudan_irr'].min()}")
print(f"ethiopia_hydro \t {results_mones['ethiopia_hydro'].max()} \t\t\t {results_emodps['ethiopia_hydro'].max()}")

Objective 		 Best MONES 		 Best EMODPS
egypt_irr 		 4.71215178 		 3.499272296771559
egypt_low_had 	 0.0 				 0.0
sudan_irr 		 0.0 				 0.0
ethiopia_hydro 	 3.8524768 			 15.125298421817458


In [407]:
obj_indexes = [0, 1, 2, 3]
directions = ["min", "min", "min", "max"]
best = [1e9, 1e9, 1e9, 0]
worst = [0, 0, 0, 1e10]

for objective in obj_indexes:
    if directions[objective] == "min":
        best[objective] = min(best[objective], results_emodps[results_emodps.columns[objective]].min())
        best[objective] = min(best[objective], results_mones[results_emodps.columns[objective]].min())
        worst[objective] = max(worst[objective], results_emodps[results_emodps.columns[objective]].max())
        worst[objective] = max(worst[objective], results_mones[results_emodps.columns[objective]].max())
    elif directions[objective] == "max":
        best[objective] = max(best[objective], results_emodps[results_emodps.columns[objective]].max())
        best[objective] = max(best[objective], results_mones[results_emodps.columns[objective]].max())
        worst[objective] = min(worst[objective], results_emodps[results_emodps.columns[objective]].min())
        worst[objective] = min(worst[objective], results_mones[results_emodps.columns[objective]].min())

print(objectives)
print("Best objective value:", best)
print("Worst objective value:", worst)

['egypt_irr', 'egypt_low_had', 'sudan_irr', 'ethiopia_hydro']
Best objective value: [3.499272296771559, 0.0, 0.0, 15.125298421817458]
Worst objective value: [18.462717128022547, 0.8833333333333333, 0.7052263225088279, 3.1468892]


In [408]:
# Normalize objectives function. The best outcome is 0, the worst is 1 for each objective value
def normalize_objs(df, worst, best):
    for i, col in enumerate(df.columns):
        df[col] = (best[i] - df[col]) / (best[i] - worst[i])
    return np.array(df)

In [409]:
from pymoo.indicators.hv import HV

# Normalise objectives
results_mones_np = normalize_objs(results_mones.__deepcopy__(), worst=worst, best=best)
results_emodps_np = normalize_objs(results_emodps.__deepcopy__(), worst=worst, best=best)

# Set reference point to be larger (worse) than all objectives. Same as in Yassin paper.
ref_point = np.array([1.2, 1.2, 1.2, 1.2])
ind = HV(ref_point=ref_point)

# Calculate Hypervolume
print("Hypervolume MONES:", ind(results_mones_np))
print("Hypervolume EMODPS:", ind(results_emodps_np))


Hypervolume MONES: 0.39129975337546846
Hypervolume EMODPS: 2.0307369609592123


In [413]:
# Epsilon

# For all points p in Paret Front. Go over all points in the compared dataset in look for smallest value e, such that p - e is smaller or equal to some p`.

# Create PF
mones_points = results_mones.to_numpy()
emodps_points = results_emodps.to_numpy()
all_points = np.concatenate([mones_points, emodps_points])

pareto_front = get_non_dominated_points(all_points, [0, 1, 2])

# Calculate additive epsilon
print("Number of Pareto front points:", len(pareto_front))
print("Number of EMODPS points:", len(emodps_points))

Number of Pareto front points: 222
Number of EMODPS points: 222


In [415]:
print(pareto_front in emodps_points)
print(emodps_points in pareto_front)

True
True
